In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Churn modelling/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = data.iloc[:,3:-1]
y = data.iloc[:,-1]

In [4]:
# create dummy variables
geography = pd.get_dummies(data = X['Geography'],drop_first=True)
gender = pd.get_dummies(data = X['Gender'],drop_first=True)

In [5]:
# concat two new columns to the X dataframe
X=pd.concat([X,geography,gender],axis=1)

In [6]:
#drop unnecessary columns
X = X.drop(['Geography','Gender'],axis=1)

## Splitting dataset

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

## Feature Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Importing ANN libraries

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [10]:
## initializing ann
classifier = Sequential()

## adding 1st hidden layer
classifier.add(Dense(units=6,activation='relu',kernel_initializer='he_uniform',input_dim = 11))

##  adding 2nd hidden layer
classifier.add(Dense(units=6,activation='relu',kernel_initializer='he_uniform'))

## adding output layer
classifier.add(Dense(units=1,activation='sigmoid',kernel_initializer='glorot_uniform'))

In [11]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [12]:
## compiling ann
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
# training ann on the train set
classifier.fit(X_train,y_train,validation_split=0.33,batch_size=10,epochs=100)

Epoch 1/100
536/536 [==============================] - 2s 2ms/step - loss: 0.5953 - accuracy: 0.7613 - val_loss: 0.5214 - val_accuracy: 0.7963
Epoch 2/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4863 - accuracy: 0.8061 - val_loss: 0.4678 - val_accuracy: 0.8065
Epoch 3/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4519 - accuracy: 0.8121 - val_loss: 0.4506 - val_accuracy: 0.8137
Epoch 4/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4369 - accuracy: 0.8166 - val_loss: 0.4418 - val_accuracy: 0.8148
Epoch 5/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4281 - accuracy: 0.8179 - val_loss: 0.4368 - val_accuracy: 0.8183
Epoch 6/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4227 - accuracy: 0.8207 - val_loss: 0.4338 - val_accuracy: 0.8217
Epoch 7/100
536/536 [==============================] - 1s 2ms/step - loss: 0.4179 - accuracy: 0.8240 - val_loss: 0.4319 - val_accuracy: 0.8251

In [14]:
# predicting test set results
y_pred = classifier.predict(X_test)
y_pred=(y_pred>0.5)

In [15]:
# make confusion_matrix and get accuracy score
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

[[1548   59]
 [ 217  176]]
0.862


## Keras Tuner : Hyperparameter Tuning

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch

In [19]:
## building the model
def build_model(hp):
  model= Sequential()
  for i in range(hp.Int('layers', 2, 20)):
    model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                                  activation='relu'))
  model.add(Dense(1, activation='linear'))

  model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss = 'binary_crossentropy', 
                metrics = ['accuracy'])
  
  return model

In [20]:
tuner = RandomSearch(
              build_model,
              objective = 'val_accuracy',
              max_trials = 5,
              executions_per_trial = 3,
              directory = 'tuner'
        )

In [21]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


## Fitting the model
The next thing to do is fitting the model. Here, tuner.search works like model.fit.

In [23]:
tuner.search(X_train, y_train, validation_split = 0.33,batch_size=10,epochs=10)

Trial 6 Complete [00h 04m 06s]
val_accuracy: 0.851066509882609

Best val_accuracy So Far: 0.851066509882609
Total elapsed time: 00h 24m 41s
INFO:tensorflow:Oracle triggered exit


In [24]:
tuner.results_summary()

Results summary
Results in tuner/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
layers: 15
units_0: 288
units_1: 480
learning_rate: 0.0001
units_2: 416
units_3: 96
units_4: 96
units_5: 320
units_6: 64
units_7: 96
units_8: 480
units_9: 352
units_10: 192
units_11: 192
units_12: 384
units_13: 224
units_14: 384
units_15: 192
units_16: 288
units_17: 384
units_18: 128
units_19: 288
Score: 0.851066509882609
Trial summary
Hyperparameters:
layers: 13
units_0: 192
units_1: 256
learning_rate: 0.0001
units_2: 352
units_3: 128
units_4: 32
units_5: 64
units_6: 480
units_7: 480
units_8: 448
units_9: 288
units_10: 32
units_11: 256
units_12: 288
units_13: 128
units_14: 352
units_15: 32
units_16: 512
units_17: 32
units_18: 384
units_19: 320
Score: 0.8113088409105936
Trial summary
Hyperparameters:
layers: 20
units_0: 512
units_1: 416
learning_rate: 0.0001
units_2: 448
units_3: 480
units_4: 128
units_5: 224
units_6: 224
units_7: 96
un

In [27]:
best_hp = tuner.get_best_hyperparameters()[0]

In [28]:
best_hp.values

{'layers': 15,
 'learning_rate': 0.0001,
 'units_0': 288,
 'units_1': 480,
 'units_10': 192,
 'units_11': 192,
 'units_12': 384,
 'units_13': 224,
 'units_14': 384,
 'units_15': 192,
 'units_16': 288,
 'units_17': 384,
 'units_18': 128,
 'units_19': 288,
 'units_2': 416,
 'units_3': 96,
 'units_4': 96,
 'units_5': 320,
 'units_6': 64,
 'units_7': 96,
 'units_8': 480,
 'units_9': 352}

In [25]:
best_model = tuner.get_best_models(num_models=1)[0]

In [29]:
best_model.evaluate(X_test,y_test)

63/63 [==============================] - 1s 5ms/step - loss: 0.4084 - accuracy: 0.8575


[0.40839266777038574, 0.8575000166893005]

In [30]:
# predicting test set results
y_pred = best_model.predict(X_test)
y_pred=(y_pred>0.5)

In [31]:
# make confusion_matrix and get accuracy score
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

[[1544   63]
 [ 222  171]]
0.8575
